In [2]:
from pathlib import Path 
import math
import pandas as pd
import pickle
disease_dir = Path("/shared/share_mala/zj2398/mimic/regression_labels_nona/")
for parquet_file in disease_dir.glob("*.parquet"):
    disease_file = pd.read_parquet(parquet_file)
    # disease_file = disease_file.dropna(subset=["numeric_value"])
    # disease_file.to_parquet(parquet_file)
    print(len(disease_file))
    # # print(disease_file)
    mask_nan = disease_file["numeric_value"].notna()
    print(sum(mask_nan))
    # print(disease_file.iloc[511934]["numeric_value"].dtype())
    # print(disease_file["numeric_value"] != float('nan'))
    # break

422757
422757
1156492
1156492
3231218
3231218
3129231
3129231


In [1]:
from pathlib import Path 
import math
import pandas as pd
import pickle
import meds_reader
from concurrent.futures import ProcessPoolExecutor, as_completed
from datetime import timedelta


disease_dir = Path("/shared/share_mala/zj2398/mimic/regression/cohort/regression_labels_nona")
task="bilirubin"
meds_path = "/user/zj2398/cache/mimic/meds_v0.6_reader"

disease_path = disease_dir/f"{task}_regression.parquet"
# df = pd.read_parquet(disease_path)

database = meds_reader.SubjectDatabase(meds_path)
# print(database[10000032])
# print(disease_file)
# Ensure types
for parquet_file in disease_dir.glob("*.parquet"):
    print(parquet_file.name)
    df = pd.read_parquet(parquet_file)
    # print(f"parquet_file {len(df)}")
    df["patient_id"] = df["patient_id"].astype(int)
    df["time"] = pd.to_datetime(df["time"], errors="coerce")

    # 2) Build unique patient list from the dataframe (so we only fetch what we need)
    all_pids = df["patient_id"].unique().tolist()

    # --- Worker function: open SubjectDatabase inside the process, compute first non-None event time per patient ---
    def first_event_times_for_chunk(pids_chunk, meds_path_str):
        from datetime import datetime
        import meds_reader as _mr

        db = _mr.SubjectDatabase(meds_path_str)

        out = {}
        for pid in pids_chunk:
            try:
                subj = db[pid]
            except KeyError:
                # no subject -> skip
                continue
            first_t = None
            for ev in subj.events:
                # ev.time can be None; keep the earliest non-None
                if ev.time is not None and ev.code != "MEDS_BIRTH":
                    first_t = ev.time
                    break
                    # if (first_t is None) or (ev.time < first_t):
                    #     first_t = ev.time
            if first_t is not None:
                out[pid] = pd.Timestamp(first_t)  # normalize to pandas Timestamp
        return out

    # 3) Parallelize over patient chunks (each worker builds its own SubjectDatabase handle)
    def chunked(iterable, size):
        for i in range(0, len(iterable), size):
            yield iterable[i:i+size]

    pid_chunks = list(chunked(all_pids, 1000))
    pid_to_first = {}

    with ProcessPoolExecutor(max_workers=128) as ex:
        futs = [ex.submit(first_event_times_for_chunk, ch, str(meds_path)) for ch in pid_chunks]
        for fut in as_completed(futs):
            pid_to_first.update(fut.result())

    # 4) Map first event times back to the dataframe
    df["first_event_time"] = df["patient_id"].map(pid_to_first)

    # Drop rows for patients with no valid first event time
    df = df.dropna(subset=["first_event_time"])

    # 5) Keep only rows within the first year after the first event (and not earlier)
    observation_window = pd.Timedelta(days=0)
    mask = df["time"] >= (df["first_event_time"]+observation_window)
    # print(mask,sum(mask))
    # print(df)
    filtered = df.loc[mask, ["patient_id", "time", "numeric_value", "unit"]]  # keep any columns you want
    # print(filtered)
    # print(filtered.groupby("patient_id",group_keys=False))
    filtered = filtered.groupby("patient_id",group_keys=False).sample(n=1,random_state=42)
    filtered = filtered.sort_values(["patient_id", "time"])
    filtered = filtered.rename(columns={
    "patient_id": "subject_id",
    "time": "prediction_time",
    "numeric_value":"numerical_value"
    })
    print(filtered)
    # (Optional) Save
    # # filtered.to_parquet("/path/to/filtered.parquet", index=False)
    store_path = "/shared/share_mala/zj2398/mimic/regression/cohort/regression_labels_random_drop/"+ parquet_file.name
    filtered.to_parquet(store_path)

    print(f"Kept {len(filtered):,} of {len(df):,} rows within {observation_window} of first event.")
    # break
# print(disease_file.iloc[511934]["numeric_value"].dtype())
# print(disease_file["numeric_value"] != float('nan'))
# break

pao2_regression.parquet
        subject_id     prediction_time  numerical_value   unit
0         10000935 2187-10-22 15:42:00             86.0  mm Hg
18        10001884 2131-01-13 02:31:00             69.0  mm Hg
21        10002013 2160-05-18 09:23:00            441.0  mm Hg
34        10002155 2129-08-04 12:27:00             89.0  mm Hg
43        10002348 2112-12-05 03:52:00            100.0  mm Hg
...            ...                 ...              ...    ...
511917    19999442 2148-11-19 17:15:00            131.0  mm Hg
511921    19999625 2139-10-10 17:31:00             71.0  mm Hg
511922    19999828 2149-01-08 10:06:00             77.0  mm Hg
511927    19999840 2164-09-13 02:50:00             86.0  mm Hg
511937    19999987 2145-11-04 07:20:00             69.0  mm Hg

[46953 rows x 4 columns]
Kept 46,953 of 422,715 rows within 0 days 00:00:00 of first event.
bilirubin_regression.parquet
         subject_id     prediction_time  numerical_value   unit
9          10000032 2180-07-25 07:

In [2]:
import pandas as pd
pred_path = "/shared/share_mala/zj2398/mimic/regression/motor/results/pao2/motor/test_predictions.parquet"
pred_reuslt = pd.read_parquet(pred_path)
print(pred_reuslt)

      subject_id     prediction_time  y_true      y_pred    residual
0       10003637 2150-05-14 18:38:00    29.0  141.474676 -112.474676
1       10005866 2148-03-10 16:14:00    87.0  202.922773 -115.922773
2       10006053 2111-11-14 00:00:00   137.0  160.296174  -23.296174
3       10013097 2135-04-29 17:54:00   142.0  226.328414  -84.328414
4       10013419 2168-08-25 11:10:00   106.0  158.802771  -52.802771
...          ...                 ...     ...         ...         ...
4747    19990821 2143-03-08 13:36:00    79.0  110.636532  -31.636532
4748    19993726 2166-09-20 09:15:00   226.0  122.806050  103.193950
4749    19995127 2138-06-10 01:08:00    69.0  128.173797  -59.173797
4750    19997886 2186-12-09 21:53:00   166.0  122.180077   43.819923
4751    19998330 2178-10-22 06:19:00    90.0  106.426063  -16.426063

[4752 rows x 5 columns]


In [ ]:
from pathlib import Path
import pandas as pd
import pickle as pl
import numpy as np

    
def find_overlapping_and_unique_rows(df1, df2, columns):
    """
    Find overlapping and unique rows between two DataFrames based on specified columns.
    """
    
    # Find overlapping rows (inner join)
    overlapping = df1.merge(df2[columns], on=columns, how='inner')
    
    # Find unique rows in df1 (left only)
    df1_with_indicator = df1.merge(df2[columns], on=columns, how='left', indicator=True)
    unique_df1 = df1_with_indicator[df1_with_indicator['_merge'] == 'left_only'].drop('_merge', axis=1)
    
    # Find unique rows in df2 (right only)
    df2_with_indicator = df2.merge(df1[columns], on=columns, how='left', indicator=True)
    unique_df2 = df2_with_indicator[df2_with_indicator['_merge'] == 'left_only'].drop('_merge', axis=1)
    
    return overlapping, unique_df1, unique_df2


feature_dir = Path("/shared/share_mala/zj2398/mimic/regression/motor/features")
label_dir = Path("/shared/share_mala/zj2398/mimic/regression/cohort/regression_labels_1_month")
task = "bilirubin"
feature_path = feature_dir/f"{task}_motor.pkl"
with open(feature_path,"rb") as f:
    feature = pl.load(f)
# df = pd.DataFrame(list(feature.items()), columns=['key', 'value'])

# print(feature)
# feature_df = pd.DataFrame({
#         "subject_id": feature['subject_ids'],
#         "prediction_time":feature['feature_times'],
#         "features": list(feature['features'])
#     })
# # df = pd.DataFrame([feature])

for task in ["bilirubin","creatinine","pao2","platelets"]:
    print(task)
    label_path = label_dir/task/f"{task}.parquet"
    label_df = pd.read_parquet(label_path)
    print(label_df)

# print(feature_df)
# print(label_df)

# overlapping,unique1,unique2 = find_overlapping_and_unique_rows(feature_df, label_df, ["subject_id","prediction_time"])
# print(f"overlapping features {overlapping}")
# print(f"unique df1 {unique1}")
# print(f"unique df2 {unique2}")

bilirubin
         subject_id     prediction_time  target_value   unit
1          10000032 2180-05-06 23:16:00           1.6  mg/dL
16         10000084 2160-12-28 05:02:00           0.6  mg/dL
18         10000117 2176-02-08 22:43:00           1.1  mg/dL
25         10000285 2161-11-08 17:44:00           0.7  mg/dL
27         10000560 2189-06-26 14:30:00           0.3  mg/dL
...             ...                 ...           ...    ...
1174777    19999625 2138-10-06 15:21:00           0.7  mg/dL
1174779    19999636 2187-09-15 19:28:00           0.5  mg/dL
1174784    19999784 2119-07-31 09:56:00           0.3  mg/dL
1174891    19999828 2149-01-08 10:47:00           0.4  mg/dL
1174896    19999840 2164-09-12 09:10:00           0.8  mg/dL

[95607 rows x 4 columns]
creatinine
         subject_id     prediction_time  target_value   unit
1          10000032 2180-05-06 23:16:00           0.3  mg/dL
24         10000084 2160-12-27 20:13:00           0.8  mg/dL
27         10000117 2176-02-08 22:43:0

In [3]:
import numpy as np
print(np.logspace(-3, 3, 13))

[1.00000000e-03 3.16227766e-03 1.00000000e-02 3.16227766e-02
 1.00000000e-01 3.16227766e-01 1.00000000e+00 3.16227766e+00
 1.00000000e+01 3.16227766e+01 1.00000000e+02 3.16227766e+02
 1.00000000e+03]
